In [3]:
import numpy as np
import matplotlib.pyplot as plt
import test
%matplotlib tk

import ilfft_interp
# Example to plot the bases of the example lattices. Each point in the graph represent
# A chebyshev basis function

#One quadrant of a (nearly) regular hexagon
U = ilfft_interp.HexLattice(2)
U.plot_basis()

#One quadrant of the L_1 ball in 2d
U = ilfft_interp.PadLattice(16)
U.plot_basis()

#one quadrant of a rhombic dodecahedron, which is a stellation of an octahedron
U = ilfft_interp.BCCLattice(7)
U.plot_basis()

#one quadrant of a truncated octahedron
U = ilfft_interp.FCCLattice(6)
U.plot_basis()

U = ilfft_interp.OctLattice(8)
U.plot_basis()

4.752157956577029
4.752157956577029
7.63560865361542
5.411474127809771
5.4114741278097735
1.4142135623730951
4.791287847477922
1.4142135623730951
1.0000000000000004


In [175]:

U = ilfft_interp.OctLattice(6)
p = U.get_points()

plt.scatter(p[:, 0], p[:, 1])
plt.show()

4.752157956577029
4.752157956577029
7.63560865361542
5.411474127809771
5.4114741278097735
1.4142135623730951
4.791287847477922
1.4142135623730951
1.0000000000000004


In [79]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.scale import FuncScale
import test
from scipy.special import roots_legendre
import ilfft_interp as bf
import mod_fejer as mf
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
%matplotlib tk

# Compare integration accuracy of the Padua and Hex lattices as well as Gauss-Legendre quadrature.
# Hex performs best in some middling resolution regimes, but GL is safer.
x_0 = -.2
y_0 = .0
leg_res = 400
x, w = roots_legendre(leg_res)
W = np.outer(w, w)
X, Y = np.meshgrid(x, x)
t_res = 600
t = np.linspace(0, 2*np.pi, t_res)[:-1]

f = lambda x, y, theta: 1/(1 + 50*((np.cos(theta)*x + np.sin(theta)*y - x_0)**2 + (np.cos(theta)*y - np.sin(theta)*x - y_0)**2))
I = [np.sum(f(X, Y, theta)*W) for theta in t]

pad_pcount = []
pad_error  = []
for i in range(2, 140, 5):
    U = bf.PadLattice(i)
    val = 0
    for j, theta in enumerate(t):
        U.eval_func(lambda x, y: f(x, y, theta))
        val += np.abs(I[j] - U.get_integral())/t_res
    pad_pcount.append(U.get_point_count())
    pad_error.append(val)
    
hex_pcount = []
hex_error  = []
for i in range(1, 24):
    print(i)
    val = 0
    U = bf.HexLattice(i)
    for j, theta in enumerate(t):
        U.eval_func(lambda x, y: f(x, y, theta))
        val += np.abs(I[j] - U.get_integral())/t_res
    hex_pcount.append(U.get_point_count())
    hex_error.append(val)

mf_pcount = []
mf_error  = []
for i in range(1, 20):
    print(i)
    U = mf.mod_fejer_hex(i)
    val = 0
    for j, theta in enumerate(t):
        val += np.abs(I[j] - np.sum(U.stencil*(f(U.X, U.Y, theta))))/t_res
    mf_pcount.append(U.get_point_count())
    mf_error.append(val)
    
leg_pcount = []
leg_error  = []
for i in range(2, 100, 10):
    x, w = roots_legendre(i)
    W = np.outer(w, w)
    val = 0
    X, Y = np.meshgrid(x, x)
    for j, theta in enumerate(t):
        val += np.abs(I[j] - np.sum(f(X, Y, theta)*W))/t_res
    leg_pcount.append(i**2)
    leg_error.append(val)
    
plt.semilogy(np.sqrt(np.array(pad_pcount)), pad_error, 'b-o')
plt.semilogy(np.sqrt(np.array(hex_pcount)), hex_error, 'r--x')
#plt.semilogy(np.sqrt(np.array(mf_pcount)), mf_error)
plt.semilogy(np.sqrt(np.array(leg_pcount)), leg_error, 'k-')
plt.semilogy(np.sqrt((7/8)*np.array(leg_pcount)), leg_error, 'k--')
#plt.xscale("function", functions=(lambda x: np.sqrt(x), lambda x: x**2))
#plt.xticks([100, 400, 1600, 3600, 6400, 10000, 14400])
plt.legend(["Padua Points", "Hex Points", "Tensor Product Gauss-Legendre"])
plt.gca().xaxis.set_major_formatter('${x:.0f}^2$')
plt.show()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [1]:
import numpy as np
import numpy.linalg as lin
from numpy.random import normal
import matplotlib.pyplot as plt
import test
from scipy.special import roots_legendre
import ilfft_interp as bf
%matplotlib tk

# Compare integration accuracy of the BCC and FCC lattices as well as Gauss-Legendre quadrature.
compiled_results = []

rot_list = []
N = 20
for i in range(N):
    #Draw a set of three random orthogonal unit vectors
    M = normal(size=(3,3))
    M[0] /= lin.norm(M[0])
    M[1] -= (M[0] @ M[1])*M[0]
    M[1] /= lin.norm(M[1])
    M[2] -= (M[0] @ M[2])*M[0]
    M[2] -= (M[1] @ M[2])*M[1]
    M[2] /= lin.norm(M[2])
    rot_list.append(M)

def xyz_transform(x, y, z, m):
    return tuple([x*v[0] + y*v[1] + z*v[2] for v in m])
    
x_0 = -.3
y_0 = .0
z_0 = .0
leg_res = 90
x, w = roots_legendre(leg_res)
W = np.einsum("i, j, k", w, w, w)
X, Y, Z = np.meshgrid(x, x, x)

func_list = [lambda x, y, z: np.exp(-(45*(x_0 - x)**2 + 45*(y_0 - y)**2 + 45*(z_0 - z)**2)),
             lambda x, y, z: 1/(1 + (30*(x_0 - x)**2 + 30*(y_0 - y)**2 + 30*(z_0 - z)**2)),
            # lambda x, y, z: np.exp(-1/(25*(x_0 - x)**2 + 25*(y_0 - y)**2 + 25*(z_0 - z)**2))
            ]

for f in func_list:
    
    #f = lambda x, y, z: 1/(1 + (25*(x_0 - x)**2 + 25*(y_0 - y)**2 + 25*(z_0 - z)**2))
    I = [np.sum(f(*xyz_transform(X, Y, Z, M))*W) for M in rot_list]
    compiled_results.append(dict())

    BCC_pcount = []
    BCC_error  = []
    for i in range(1, 50, 1):
        print(i)
        U = bf.BCCLattice(i)
        tmp_err = []
        for j, m in enumerate(rot_list):
            f_tmp = lambda x, y, z: f(*xyz_transform(x, y, z, m))
            U.eval_func(f_tmp)
            tmp_err.append(np.abs(I[j] - U.get_integral())/N)

        BCC_pcount.append(U.get_point_count())
        BCC_error.append(np.sum(tmp_err))

    FCC_pcount = []
    FCC_error  = []
    for i in range(1, 50, 1):
        print(i)
        U = bf.FCCLattice(i)
        tmp_err = []
        for j, m in enumerate(rot_list):
            f_tmp = lambda x, y, z: f(*xyz_transform(x, y, z, m))
            U.eval_func(f_tmp)
            tmp_err.append(np.abs(I[j] - U.get_integral())/N)

        FCC_pcount.append(U.get_point_count())
        FCC_error.append(np.sum(tmp_err))

    leg_pcount = []
    leg_error  = []
    for i in range(2, 73, 4):
        print(i)
        x, w = roots_legendre(i)
        W = np.einsum("i,j,k", w, w, w)
        X, Y, Z = np.meshgrid(x, x, x)
        tmp_err = []
        for j, m in enumerate(rot_list):
            f_tmp = lambda x, y, z: f(*xyz_transform(x, y, z, m))
            tmp_err.append(np.abs(I[j] - np.sum(f_tmp(X, Y, Z)*W))/N)
        leg_pcount.append(i**3)
        leg_error.append(np.sum(tmp_err))
    compiled_results[-1]['bp'] = BCC_pcount
    compiled_results[-1]['be'] = BCC_error
    compiled_results[-1]['fp'] = FCC_pcount
    compiled_results[-1]['fe'] = FCC_error
    compiled_results[-1]['lp'] = leg_pcount
    compiled_results[-1]['le'] = leg_error
    
plt.semilogy((np.array(BCC_pcount))**(1/3), BCC_error)
plt.semilogy((np.array(FCC_pcount))**(1/3), FCC_error)
plt.semilogy((np.array(leg_pcount))**(1/3), leg_error)
plt.show()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
2
6
10
14
18
22
26
30
34
38
42
46
50
54
58
62
66
70
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
2
6
10
14
18
22
26
30
34
38
42
46
50
54
58
62
66
70


In [174]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

for i in range(2):
    ax[i].semilogy((np.array(compiled_results[i]['bp']))**(1/3), compiled_results[i]['be'], 'm-s')
    ax[i].semilogy((np.array(compiled_results[i]['fp']))**(1/3), compiled_results[i]['fe'], 'b-x')
    ax[i].semilogy((np.array(compiled_results[i]['lp']))**(1/3), compiled_results[i]['le'], 'k-')
    ax[i].semilogy((.71*np.array(compiled_results[i]['lp']))**(1/3), compiled_results[i]['le'], 'k--')
    if i == 0:
        ax[i].legend(["BCC points", "FCC Points", "Gauss-Legendre"])
    ax[i].xaxis.set_major_formatter('${x:.0f}^3$')
    
plt.show()

In [160]:
plt.semilogy((np.array(BCC_pcount))**(1/3), BCC_error, 'r-s')
plt.semilogy((np.array(FCC_pcount))**(1/3), FCC_error, 'b-o')
plt.semilogy((np.array(leg_pcount))**(1/3), leg_error, 'k-')
plt.semilogy((.71*np.array(leg_pcount))**(1/3), leg_error, 'k--')
plt.legend(["BCC Points", "FCC Points", "Tensor Product Gauss-Legendre", "Theoretical BCC"])
plt.gca().xaxis.set_major_formatter('${x:.0f}^3$')
plt.show()
plt.show()

In [77]:
## import numpy as np
import numpy.linalg as lin
import ilfft_interp as bf
import matplotlib.pyplot as plt
%matplotlib tk

x_0 = .1
y_0 = .2
f  = lambda x, y: 1/(1 + 25*(x_0 - x)**2 + 25*(y_0 - y)**2)
df = lambda x, y: -50*(x - x_0)/(1 + 25*(x_0 - x)**2 + 25*(y_0 - y)**2)**2

def cheb_diff(N, f, df):
    c = lambda i: 1 + ((i == 0) or (i == N))
    D = np.zeros((N + 1, N + 1))
    x = bf.cheby_points(N + 1)
    for i in range(N + 1):
        D[i,i] = -x[i]/(2*(1 - x[i]**2)) if (i != 0) and (i != N) else ((-1)**(i != 0))*(2*N**2 + 1)/6
        for j in range(i):
            D[i, j] = (c(i)/c(j))*((-1)**(i + j))/(x[i] - x[j])
            D[j, i] = (c(j)/c(i))*((-1)**(i + j))/(x[j] - x[i])
    X, Y = np.meshgrid(x, x)
    Z = f(X, Y)
    for i in range(N + 1):
        Z[i] = D @ Z[i]
    Z -= df(X, Y)
    return Z

cheb_pcount = []
cheb_error  = []
for i in range(2, 200):
    Z = cheb_diff(i, f, df)
    Z[...] = Z**2
    Z *= 4/(i**2)
    Z[0] *= .5
    Z[-1] *= .5
    Z[:, 0] *= .5
    Z[:, -1] *= .5
    cheb_pcount.append((i + 1)**2)
    cheb_error.append(np.sqrt(np.sum(Z)))

hex_pcount = []
hex_error = []
for i in range(1, 50):
    print(i)
    U = bf.HexLattice(i)
    U.eval_func(f)
    U.transform()
    U.get_deriv(0)
    U.inverse_transform()
    g = [L.copy() for L in U.grids]
    U.eval_func(df)
    for i in range(len(U.grids)):
        U.grids[i] -= g[i]
        U.grids[i][...] = U.grids[i]**2
    U.apply_chebyshev_weights()
    hex_pcount.append(U.get_point_count())
    hex_error.append(np.sqrt(sum([np.sum(L) for L in U.grids])))

pad_pcount = []
pad_error = []
for i in range(2, 200, 5):
    print(i)
    U = bf.PadLattice(i)
    U.eval_func(f)
    U.transform()
    U.get_deriv(0)
    U.inverse_transform()
    g = [L.copy() for L in U.grids]
    U.eval_func(df)
    for i in range(len(U.grids)):
        U.grids[i] -= g[i]
        U.grids[i][...] = U.grids[i]**2
    U.apply_chebyshev_weights()
    pad_pcount.append(U.get_point_count())
    pad_error.append(np.sqrt(sum([np.sum(L) for L in U.grids])))

plt.semilogy(np.sqrt(np.array(cheb_pcount)), cheb_error)
plt.semilogy(np.sqrt(np.array(hex_pcount)), hex_error)
plt.semilogy(np.sqrt(np.array(pad_pcount)), pad_error)
plt.show()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
2
7
12
17
22
27
32
37
42
47
52
57
62
67
72
77
82
87
92
97
102
107
112
117
122
127
132
137
142
147
152


KeyboardInterrupt: 

In [5]:
import test
import ilfft_interp as bf
import numpy as np

U = bf.PadLattice(15)
test.test_int_accuracy(U)

(0, 0) 4.0 4.0
(0, 1) 0.0 8.326672684688674e-17
(0, 2) -1.3333333333333333 -1.3333333333333335
(0, 3) 0.0 2.7755575615628914e-17
(0, 4) -0.26666666666666666 -0.2666666666666666
(0, 5) 0.0 4.2760933682828295e-16
(0, 6) -0.11428571428571428 -0.11428571428571385
(0, 7) 0.0 -6.277727480437707e-16
(1, 0) 0.0 4.215378046623641e-16
(1, 1) 0.0 -5.204170427930421e-18
(1, 2) 0.0 -9.280770596475918e-17
(1, 3) 0.0 1.1709383462843448e-17
(1, 4) 0.0 -2.2985086056692694e-17
(1, 5) 0.0 1.9949319973733282e-17
(1, 6) 0.0 5.637851296924623e-18
(1, 7) 0.0 -3.926737127104592e-18
(2, 0) -1.3333333333333333 -1.333333333333334
(2, 1) 0.0 -5.551115123125783e-17
(2, 2) 0.4444444444444444 0.4444444444444447
(2, 3) 0.0 0.0
(2, 4) 0.08888888888888888 0.08888888888888896
(2, 5) 0.0 -1.734723475976807e-16
(2, 6) 0.03809523809523809 0.03809523809523793
(2, 7) 0.0 2.0794867030555546e-16
(3, 0) 0.0 1.5439038936193583e-16
(3, 1) 0.0 -1.3877787807814457e-17
(3, 2) 0.0 -1.0321604682062002e-16
(3, 3) 0.0 3.903127820947816e

In [4]:
U.coefs[1]

array([[0.25, 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ]])

In [24]:
import matplotlib.pyplot as plt
%matplotlib tk
t = np.linspace(-np.pi, np.pi, 100)
r_1 = np.cos(t)
r_2 = np.sin(t)
plt.plot(r_1*np.cos(t), r_1*np.sin(t))
plt.plot(r_2*np.cos(t), r_2*np.sin(t))
plt.plot(-r_1*np.cos(t), -r_1*np.sin(t))
plt.plot(-r_2*np.cos(t), -r_2*np.sin(t))
plt.show()

In [2]:
test.test_invertibility(U)

7.938288718631387e-16
1.061809488300179e-15


[array([[0.58374207, 0.28307268, 0.25346551, 0.62228914, 0.44599955,
         0.53398137, 0.22905578, 0.43440956],
        [0.17824966, 0.60534404, 0.98641624, 0.62786555, 0.10256071,
         0.51616633, 0.15543663, 0.76295868],
        [0.52679034, 0.03575601, 0.47406434, 0.44788915, 0.25789924,
         0.21735807, 0.47829774, 0.60821494],
        [0.03680063, 0.31923046, 0.27034548, 0.72629748, 0.46232116,
         0.42516746, 0.11934163, 0.00425909],
        [0.09176197, 0.9879996 , 0.34790996, 0.74243541, 0.09756071,
         0.20533519, 0.02364514, 0.30587048],
        [0.80549623, 0.92944698, 0.31227806, 0.87748313, 0.24381142,
         0.61005816, 0.92047699, 0.29465217],
        [0.22570837, 0.60613698, 0.5667757 , 0.96317035, 0.22866489,
         0.11493784, 0.31923338, 0.02620336],
        [0.48932298, 0.11099205, 0.56332553, 0.69921852, 0.85755754,
         0.93733673, 0.28520703, 0.52837708]]),
 array([[0.78707179, 0.54242719, 0.80514197, 0.09560315, 0.4726613 ,
         

In [6]:
import ilfft_interp as bf
%matplotlib tk
U = bf.BCCLattice(6)
U.plot_basis()

In [7]:
import ilfft_interp as bf
import numpy as np
import scipy.fft as fft
import numpy.linalg as lin


N = 6
x = bf.cheby_points(4*N + 1)[1:-1]
y = bf.cheby_points(7*N + 1)[1:-1]

X, Y = np.meshgrid(x, y)
Z = np.zeros(X.shape)
for ind in np.ndindex(X.shape):
    Z[ind] = (ind[0] + ind[1])%2 == 0
Z_hat = fft.dstn(Z, type=1)

In [8]:
plt.pcolor(Z_hat**(1/4))
plt.show()

/tmp/ipykernel_14030/766191900.py:1: RuntimeWarning: invalid value encountered in power
  plt.pcolor(Z_hat**(1/4))


In [2]:
V = 

In [9]:
U.X

array([[ 0.99638492,  0.99638492,  0.99638492, ...,  0.99638492,
         0.99638492,  0.99638492],
       [ 0.9819681 ,  0.9819681 ,  0.9819681 , ...,  0.9819681 ,
         0.9819681 ,  0.9819681 ],
       [ 0.9580944 ,  0.9580944 ,  0.9580944 , ...,  0.9580944 ,
         0.9580944 ,  0.9580944 ],
       ...,
       [-0.9580944 , -0.9580944 , -0.9580944 , ..., -0.9580944 ,
        -0.9580944 , -0.9580944 ],
       [-0.9819681 , -0.9819681 , -0.9819681 , ..., -0.9819681 ,
        -0.9819681 , -0.9819681 ],
       [-0.99638492, -0.99638492, -0.99638492, ..., -0.99638492,
        -0.99638492, -0.99638492]])

In [80]:
# Three Dimensional L2 error

import numpy as np
import numpy.linalg as lin
import ilfft_interp as bf
import matplotlib.pyplot as plt
import scipy.fft as fft
from numpy.random import normal
%matplotlib tk

class results_3d:
    def __init__(self, fp, fe, cp, ce, bp, be):
        self.fp = fp
        self.fe = fe
        self.cp = cp
        self.ce = ce
        self.bp = bp
        self.be = be

V = bf.BCCLattice(60)
rot_list = []
N = 30
for i in range(N):
    #Draw a set of three random orthogonal unit vectors
    M = normal(size=(3,3))
    M[0] /= lin.norm(M[0])
    M[1] -= (M[0] @ M[1])*M[0]
    M[1] /= lin.norm(M[1])
    M[2] -= (M[0] @ M[2])*M[0]
    M[2] -= (M[1] @ M[2])*M[1]
    M[2] /= lin.norm(M[2])
    rot_list.append(M)

def xyz_transform(x, y, z, m):
    return tuple([x*v[0] + y*v[1] + z*v[2] for v in m])
    
x_0 = -.15
y_0 = .15
z_0 = .0
f_list = [ lambda x, y, z: np.exp(-3*(x_0 - x)**2 - y**2 - z**2),
    lambda x, y, z: 1/(1 + 3*((x_0 - x)**2 + (y_0 - y)**2 + (z_0 - z)**2)),
    lambda x, y, z: np.exp(-2/((x_0 - x)**2 + (y-y_0)**2 + z**2))
         ]
r_list = []
for f_i, f in enumerate(f_list):
    val_list = []
    for M in rot_list:
        V.eval_func(lambda x,y,z: f(*xyz_transform(x, y, z, M)))
        val_list.append((V.grids[0].copy(), V.grids[1].copy()))


    BCC_pcount = []
    BCC_error  = []
    for i in range(2, 20):
        print(i)
        U = bf.BCCLattice(i)
        tmp_err = []
        for j, M in enumerate(rot_list):
            f_tmp = lambda x, y, z: f(*xyz_transform(x, y, z, M))
            U.eval_func(f_tmp)
            U.transform()
            c = U.to_cartesian()
            V.coefs[0][...] = 0
            V.coefs[1][...] = 0
            V.coefs[0][:c.shape[0], :c.shape[1], :c.shape[2]] = c
            V.inverse_transform()
            V.grids[0][...] = (V.grids[0] - val_list[j][0])**2
            V.grids[1][...] = (V.grids[1] - val_list[j][1])**2
            tmp_err.append(np.sqrt(V.get_integral())/N)
        
        BCC_pcount.append(U.get_point_count())
        BCC_error.append(np.sum(tmp_err))

    FCC_error  = []
    FCC_pcount = []
    coef_check = []
    coef_list = []
    for i in range(2, 18):
        print(i)
        U = bf.FCCLattice(i)
        tmp_err = []
        for j, M in enumerate(rot_list):
            f_tmp = lambda x, y, z: f(*xyz_transform(x, y, z, M))
            U.eval_func(f_tmp)
            U.transform()
            c = U.to_cartesian()
            V.coefs[0][...] = 0
            V.coefs[1][...] = 0
            V.coefs[0][:c.shape[0], :c.shape[1], :c.shape[2]] = c
            V.inverse_transform()
            V.grids[0][...] = (V.grids[0] - val_list[j][0])**2
            V.grids[1][...] = (V.grids[1] - val_list[j][1])**2
            tmp_err.append(np.sqrt(V.get_integral())/N)
            V.grids[0][...] = val_list[j][0]
            V.grids[1][...] = val_list[j][1]
            V.transform()
        
        
        FCC_pcount.append(U.get_point_count())
        FCC_error.append(np.sum(tmp_err))   

    cart_err = []
    cart_pcount = []
    for i in range(2, 30, 3):
        print(i)
        x = bf.cheby_points(i)
        X, Y, Z = np.meshgrid(x, x, x, indexing='ij')
        tmp_err = []
        for j, M in enumerate(rot_list):
            f_tmp = lambda x, y, z: f(*xyz_transform(x, y, z, M))
            F = f_tmp(X, Y, Z)
            c = fft.dctn(F, type=1)/((F.shape[0] - 1)**3)
            c[0,...] /= 2
            c[:,0,:] /= 2
            c[:,:,0] /= 2
            c[-1,...] /= 2
            c[:,-1,:] /= 2
            c[...,-1] /= 2
            V.coefs[0][...] = 0
            V.coefs[1][...] = 0
            V.coefs[0][:c.shape[0], :c.shape[1], :c.shape[2]] = c
            V.inverse_transform()
            V.grids[0][...] = (V.grids[0] - val_list[j][0])**2
            V.grids[1][...] = (V.grids[1] - val_list[j][1])**2
            tmp_err.append(np.sqrt(V.get_integral())/N)
        
        cart_pcount.append(i**3)
        cart_err.append(np.sum(tmp_err))
    
    
    r_list.append(results_3d(FCC_pcount, FCC_error, cart_pcount, cart_err, BCC_pcount, BCC_error))

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2
5
8
11
14
17
20
23
26
29
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2
5
8
11
14
17
20
23
26
29
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2
5
8
11
14
17
20
23
26
29


In [83]:
r = r_list[2]

#plt.semilogy((np.array(r.bp))**(1/3), r.be, 'r-o')
#plt.semilogy((np.array(r.fp))**(1/3), r.fe, 'b-o')
#plt.semilogy((np.array(r.cp))**(1/3), r.ce, 'k-')
#plt.semilogy((.71*np.array(r.cp))**(1/3), r.ce, 'k--')
#plt.legend(["BCC Points", "FCC Points", "Tensor Product", "Theoretical BCC"])
#plt.gca().xaxis.set_major_formatter('${x:.0f}^3$')
#plt.show()
#plt.show()
fig, ax = plt.subplots(1, 3, figsize=(12, 4))

for i in range(3):
    ax[i].semilogy((np.array(r_list[i].bp))**(1/3), r_list[i].be, 'r-s')
    ax[i].semilogy((np.array(r_list[i].fp))**(1/3), r_list[i].fe, 'b-x')
    ax[i].semilogy((np.array(r_list[i].cp))**(1/3), r_list[i].ce, 'k-')
    ax[i].semilogy(((.71)*np.array(r_list[i].cp))**(1/3), r_list[i].ce, 'k--')
    if i == 0:
        ax[i].legend(["BCC Lattice", "FCC Lattice", "Tensor Product"])
    ax[i].xaxis.set_major_formatter('${x:.0f}^3$')
    
plt.show()

In [70]:
import numpy as np
import numpy.linalg as lin
import ilfft_interp as bf
import matplotlib.pyplot as plt
import scipy.fft as fft
from numpy.random import normal
%matplotlib tk

class results_storage:
    def __init__(self, pp, pe, cp, ce, op, oe, hp, he):
        self.pp = pp
        self.pe = pe
        self.cp = cp
        self.ce = ce
        self.op = op
        self.oe = oe
        self.hp = hp
        self.he = he
rs = []
func_list = []
func_names = []
x_0 = -.2
y_0 = .0
z_0 = .0
func_list.append(lambda x, y:np.cos(1 - (12*(x_0 - x) + 12*(y_0 - y))) + np.sin(1 - (12*x - 12*y)))
func_names.append("Entire")
func_list.append(lambda x, y:1/((2*(2.3 - x) + 2*(y_0 - y))))
func_names.append("Analytic")
func_list.append(lambda x, y: 10*np.exp(-1/(.1*(x_0 - x)**2 + .1*(y_0 - y)**2)))
func_names.append("$C_\infty$")
                 
for f in func_list:
    V = bf.PadLattice(300)
    rot_list = []
    N = 60
    for t in np.linspace(0, 2*np.pi, N)[:-1]:
        #Draw a set of three random orthogonal unit vectors
        M = np.array([[np.cos(t), np.sin(t)],[-np.sin(t), np.cos(t)]])
        rot_list.append(M)

    def xyz_transform(x, y, m):
        return tuple([x*v[0] + y*v[1] for v in m])


    val_list = []
    l2_norms = []
    for M in rot_list:
        V.eval_func(lambda x,y: f(*xyz_transform(x, y, M)))
        val_list.append((V.grids[0].copy(), V.grids[1].copy()))
        V.grids[0][...] = V.grids[0]**2
        V.grids[1][...] = V.grids[1]**2
        l2_norms.append(np.sqrt(V.get_integral()))


    pad_pcount = []
    pad_error  = []
    for i in range(20, 70, 4):
        print(i)
        U = bf.PadLattice(i)
        tmp_err = []
        for j, M in enumerate(rot_list):
            f_tmp = lambda x, y: f(*xyz_transform(x, y, M))
            U.eval_func(f_tmp)
            U.transform()
            c = U.to_cartesian()
            V.coefs[0][...] = 0
            V.coefs[1][...] = 0
            V.coefs[0][:c.shape[0], :c.shape[1]] = c
            V.inverse_transform()
            V.grids[0][...] = (V.grids[0] - val_list[j][0])**2
            V.grids[1][...] = (V.grids[1] - val_list[j][1])**2
            tmp_err.append(np.sqrt(V.get_integral())/(N*l2_norms[j]))
        
        pad_pcount.append(U.get_point_count())
        pad_error.append(np.sum(tmp_err))
    
    cart_err = []
    cart_pcount = []
    for i in range(15, 50, 3):
        print(i)
        x = bf.cheby_points(i)
        X, Y = np.meshgrid(x, x, indexing='ij')
        tmp_err = []
        for j, M in enumerate(rot_list):
            f_tmp = lambda x, y: f(*xyz_transform(x, y, M))
            F = f_tmp(X, Y)
            c = fft.dctn(F, type=1)/((F.shape[0] - 1)**2)
            c[0,...] /= 2
            c[:,0] /= 2
            c[-1,...] /= 2
            c[:,-1] /= 2
            V.coefs[0][...] = 0
            V.coefs[1][...] = 0
            V.coefs[0][:c.shape[0], :c.shape[1]] = c
            V.inverse_transform()
            V.grids[0][...] = (V.grids[0] - val_list[j][0])**2
            V.grids[1][...] = (V.grids[1] - val_list[j][1])**2
            tmp_err.append(np.sqrt(V.get_integral())/(N*l2_norms[j]))
        
        cart_pcount.append(i**2)
        cart_err.append(np.sum(tmp_err))

    hex_err = []
    hex_pcount = []
    for i in range(4, 12):
        print(i)
        U = bf.HexLattice(i)
        tmp_err = []
        for j, M in enumerate(rot_list):
            f_tmp = lambda x, y: f(*xyz_transform(x, y, M))
            U.eval_func(f_tmp)
            U.transform()
            c = U.to_cartesian()
            V.coefs[0][...] = 0
            V.coefs[1][...] = 0
            V.coefs[0][:c.shape[0], :c.shape[1]] = c
            V.inverse_transform()
            V.grids[0][...] = (V.grids[0] - val_list[j][0])**2
            V.grids[1][...] = (V.grids[1] - val_list[j][1])**2
            tmp_err.append(np.sqrt(V.get_integral())/(N*l2_norms[j]))
        
        hex_pcount.append(U.get_point_count())
        hex_err.append(np.sum(tmp_err))

    oct_err = []
    oct_pcount = []
    for i in range(6, 18):
        print(i)
        U = bf.OctLattice(i)
        tmp_err = []
        for j, M in enumerate(rot_list):
            f_tmp = lambda x, y: f(*xyz_transform(x, y, M))
            U.eval_func(f_tmp)
            U.transform()
            c = U.to_cartesian()
            V.coefs[0][...] = 0
            V.coefs[1][...] = 0
            V.coefs[0][:c.shape[0], :c.shape[1]] = c
            V.inverse_transform()
            V.grids[0][...] = (V.grids[0] - val_list[j][0])**2
            V.grids[1][...] = (V.grids[1] - val_list[j][1])**2
            tmp_err.append(np.sqrt(V.get_integral())/(N*l2_norms[j]))
        oct_pcount.append(U.get_point_count())
        oct_err.append(np.sum(tmp_err))
    rs.append(results_storage(pad_pcount, pad_error, cart_pcount, cart_err, oct_pcount, oct_err, hex_pcount, hex_err))


20
24
28
32
36
40
44
48
52
56
60
64
68
15
18
21
24
27
30
33
36
39
42
45
48
4
5
6
7
8
9
10
11
6
4.752157956577029
4.752157956577029
7.63560865361542
5.411474127809771
5.4114741278097735
1.4142135623730951
4.791287847477922
1.4142135623730951
1.0000000000000004
7
4.752157956577029
4.752157956577029
7.63560865361542
5.411474127809771
5.4114741278097735
1.4142135623730951
4.791287847477922
1.4142135623730951
1.0000000000000004
8
4.752157956577029
4.752157956577029
7.63560865361542
5.411474127809771
5.4114741278097735
1.4142135623730951
4.791287847477922
1.4142135623730951
1.0000000000000004
9
4.752157956577029
4.752157956577029
7.63560865361542
5.411474127809771
5.4114741278097735
1.4142135623730951
4.791287847477922
1.4142135623730951
1.0000000000000004
10
4.752157956577029
4.752157956577029
7.63560865361542
5.411474127809771
5.4114741278097735
1.4142135623730951
4.791287847477922
1.4142135623730951
1.0000000000000004
11
4.752157956577029
4.752157956577029
7.63560865361542
5.4114741278097

In [81]:
fig, ax = plt.subplots(1, 3, figsize=(12, 4))

for i in range(3):
    ax[i].semilogy((np.array(rs[i].pp))**(1/2), rs[i].pe, 'm-s')
    ax[i].semilogy((np.array(rs[i].hp))**(1/2), rs[i].he, 'b-x')
    ax[i].semilogy((np.array(rs[i].op))**(1/2), rs[i].oe, 'r-o')
    ax[i].semilogy((np.array(rs[i].cp))**(1/2), rs[i].ce, 'k-')
    ax[i].semilogy(((7/8)*np.array(rs[i].cp))**(1/2), rs[i].ce, 'k--')
    if i == 0:
        ax[i].legend(["Padua Points", "Hex Points", "Seven-Point Composite", "Tensor Product"])
    ax[i].xaxis.set_major_formatter('${x:.0f}^2$')
    
plt.show()

In [10]:
import ilfft_interp as bf

U = bf.FCCLattice(10)
f = lambda x, y, z: np.exp(5*x + 5*y + 5*z)

U.eval_func(f)

In [11]:
U.transform()

In [12]:
x = U.to_cartesian()

In [30]:
plt.imshow(np.log10(x[:,:,18]))

/tmp/ipykernel_1206544/238283603.py:1: RuntimeWarning: divide by zero encountered in log10
  plt.imshow(np.log10(x[:,:,18]))


In [5]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib tk

#plt.imshow(np.log10(np.abs(coef_check[5][:,:,0])))
plt.imshow(np.log10((coef_list[-5][:,:,0])))

/tmp/ipykernel_1932945/2396804763.py:6: RuntimeWarning: divide by zero encountered in log10
  plt.imshow(np.log10((coef_list[-5][:,:,0])))
/tmp/ipykernel_1932945/2396804763.py:6: RuntimeWarning: invalid value encountered in log10
  plt.imshow(np.log10((coef_list[-5][:,:,0])))


In [1]:
import ilfft_interp as bf
U = bf.FCCLattice(10)
U.to_cartesian()

type_1 [20  0  0]
type_1 [20  0  1]
type_1 [20  0  2]
type_1 [20  0  3]
type_1 [20  0  4]
type_1 [20  0  5]
type_1 [20  0  6]
type_1 [20  0  7]
type_1 [20  0  8]
type_1 [20  0  9]
type_1 [20  1  0]
type_1 [20  1  1]
type_1 [20  1  2]
type_1 [20  1  3]
type_1 [20  1  4]
type_1 [20  1  5]
type_1 [20  1  6]
type_1 [20  1  7]
type_1 [20  1  8]
type_3 [20  1  9] [ 0 19 11]
type_1 [20  2  0]
type_1 [20  2  1]
type_1 [20  2  2]
type_1 [20  2  3]
type_1 [20  2  4]
type_1 [20  2  5]
type_1 [20  2  6]
type_1 [20  2  7]
type_3 [20  2  8] [ 0 18 12]
type_2 [ 0 18 11]
type_1 [20  3  0]
type_1 [20  3  1]
type_1 [20  3  2]
type_1 [20  3  3]
type_1 [20  3  4]
type_1 [20  3  5]
type_1 [20  3  6]
type_3 [20  3  7] [ 0 17 13]
type_2 [ 0 17 12]
type_2 [ 0 17 11]
type_1 [20  4  0]
type_1 [20  4  1]
type_1 [20  4  2]
type_1 [20  4  3]
type_1 [20  4  4]
type_1 [20  4  5]
type_3 [20  4  6] [ 0 16 14]
type_2 [ 0 16 13]
type_2 [ 0 16 12]
type_2 [ 0 16 11]
type_1 [20  5  0]
type_1 [20  5  1]
type_1 [20  5  2]
ty

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [41]:
%matplotlib tk
U.plot_basis()

In [23]:
import ilfft_interp as bf

U = bf.OctLattice(10)
U.eval_func(lambda x, y: 1./((4 + (x) + (y))))

4.752157956577029
4.752157956577029
7.63560865361542
5.411474127809771
5.4114741278097735
1.4142135623730951
4.791287847477922
1.4142135623730951
1.0000000000000004


In [24]:
A_list = [x.copy() for x in U.grids]
U.transform()
A = U.to_cartesian()

In [25]:
import numpy.linalg as lin
for i in U.M_list:
    print(lin.cond(i))

4.752157956577029
4.752157956577029
7.63560865361542
5.411474127809771
5.4114741278097735
1.4142135623730951
4.791287847477922
1.4142135623730951
1.0000000000000004


In [26]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib tk
plt.imshow(np.log10(np.abs(A)))

/tmp/ipykernel_199367/884211694.py:4: RuntimeWarning: divide by zero encountered in log10
  plt.imshow(np.log10(np.abs(A)))


In [207]:
import ilfft_interp as bf
import numpy as np
import scipy.fft as fft
%matplotlib tk

#f = lambda x, y: np.exp(-20*((x - .5)**2 + y**2))
f = lambda x, y: 1/(1 + 5*((x - .5)**2 + y**2))
x = bf.cheby_points(100)
X, Y = np.meshgrid(x, x)
coef = np.zeros(X.shape) - 16
for t in np.linspace(0, 2*np.pi, 30)[:-1]:
    Z = f(np.cos(t)*X + np.sin(t)*Y, np.cos(t)*Y - np.sin(t)*X)
    Z_hat = fft.dctn(Z, type=1)
    Z_hat[0, :] /= 2
    Z_hat[-1, :] /= 2
    Z_hat[:, 0] /= 2
    Z_hat[:, -1] /= 2
    coef = np.maximum(coef, np.log10(abs(Z_hat)))
h = plt.contourf(coef)
plt.colorbar()
plt.show()

/tmp/ipykernel_489289/2080902099.py:18: RuntimeWarning: divide by zero encountered in log10
  coef = np.maximum(coef, np.log10(abs(Z_hat)))


In [22]:
U.grids[0]

array([[0.5, 0.5],
       [0.5, 0.5]])

In [23]:
A_list[0]

array([[0.5, 0.5],
       [0.5, 0.5]])

In [27]:
import numpy as np
ind = (0,9)
A = U.test_aliasing(ind).T
A /= A[0,0]
print(np.round(A, 2))
print(U.M_list[U.get_alias_case(ind)])
print(U.get_alias_case(ind))

[[ 1.    0.    0.    0.  ]
 [ 0.25  0.25 -0.25 -0.25]
 [ 0.25 -0.25  0.25 -0.25]
 [ 0.25  0.25  0.25  0.25]]
[[ 1.  0.  0.  0.]
 [ 1.  1. -1. -1.]
 [ 1. -1.  1. -1.]
 [ 1.  1.  1.  1.]]
6


In [17]:
U.plot_basis()

In [30]:
c_list_1[-1]

array([[ 2.19724688e-01, -3.88429374e-18, -1.29891595e-01, ...,
         7.08180380e-11, -6.82864538e-19, -3.80865256e-11],
       [-8.71363102e-02,  2.04077663e-18,  7.49864427e-02, ...,
        -4.29485849e-11,  3.30346483e-19,  2.33582369e-11],
       [-9.93242243e-02,  7.92030412e-18,  9.83430639e-02, ...,
        -1.17180548e-10,  2.04636147e-18,  0.00000000e+00],
       ...,
       [-3.37947101e-10, -1.06907812e-18,  6.62324029e-10, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.61629272e-10, -2.95631336e-18,  3.15078977e-10, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.66076194e-10,  2.57086902e-19, -3.25050986e-10, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [34]:
c_list_1[-1][:31, :31] - c_list_2[-1]

array([[ 2.32441577e-09, -3.88429374e-18, -6.54920107e-09,
         9.99028289e-18,  1.38792067e-08, -4.56325404e-18,
        -3.29422000e-08, -4.39064763e-18,  8.02118429e-08,
         4.82474766e-18,  9.51055640e-12,  4.12451091e-18,
        -8.02328558e-08, -9.26732046e-18,  3.29693251e-08,
         1.10839344e-17, -1.39169209e-08, -5.88563939e-18,
         6.60239190e-09,  1.28369537e-18, -4.72259489e-09,
         5.74014428e-19,  6.64834591e-09,  9.24447308e-18,
        -1.40075802e-08,  3.13707090e-18,  3.31022213e-08,
        -4.27358487e-19, -8.04050982e-08,  8.79709865e-19,
         2.20988332e-10],
       [-2.45475988e-09,  2.04077663e-18,  6.01362650e-09,
        -5.75282779e-18, -1.03324004e-08,  2.51230898e-18,
         2.17529931e-08,  4.05646874e-19, -5.04178131e-08,
        -1.00950512e-18, -1.85870403e-11, -3.46428197e-18,
         5.04603972e-08,  3.22416294e-18, -2.18128104e-08,
         6.52224223e-19,  1.04242800e-08, -2.36113542e-18,
        -6.15734332e-09,  1.98

In [129]:
U = bf.PadLattice(20)

In [130]:
U.get_point_count()

210

In [22]:
import ilfft_interp as bf
import matplotlib.pyplot as plt
import numpy as np

%matplotlib tk

U = bf.OctLattice(7)
p = np.array(U.get_points())
s_x = np.array([-1., 1, 1, -1, -1])
s_y = np.array([-1., -1, 1, 1, -1])
plt.scatter(p[:, 0], p[:, 1], c='k', marker='x')
plt.plot(s_x, s_y, 'k--')
plt.plot(np.sin(s_x*np.pi/12),np.sin(s_y*np.pi/12),'k:')
plt.gca().axis("equal")
plt.gca().set_axis_off()
plt.show()

4.752157956577029
4.752157956577029
7.63560865361542
5.411474127809771
5.4114741278097735
1.4142135623730951
4.791287847477922
1.4142135623730951
1.0000000000000004


[ 1.          0.92387953  0.70710678  0.38268343 -0.         -0.38268343
 -0.70710678 -0.92387953 -1.          1.          0.92387953  0.70710678
  0.38268343 -0.         -0.38268343 -0.70710678 -0.92387953 -1.
  1.          0.92387953  0.70710678  0.38268343 -0.         -0.38268343
 -0.70710678 -0.92387953 -1.          1.          0.92387953  0.70710678
  0.38268343 -0.         -0.38268343 -0.70710678 -0.92387953 -1.
  1.          0.92387953  0.70710678  0.38268343 -0.         -0.38268343
 -0.70710678 -0.92387953 -1.          1.          0.92387953  0.70710678
  0.38268343 -0.         -0.38268343 -0.70710678 -0.92387953 -1.
  1.          0.92387953  0.70710678  0.38268343 -0.         -0.38268343
 -0.70710678 -0.92387953 -1.          1.          0.92387953  0.70710678
  0.38268343 -0.         -0.38268343 -0.70710678 -0.92387953 -1.
  1.          0.92387953  0.70710678  0.38268343 -0.         -0.38268343
 -0.70710678 -0.92387953 -1.          0.98078528  0.83146961  0.55557023
  0.195090

In [67]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.fft as fft
%matplotlib tk

fig, axs = plt.subplots(1, 3)

In [68]:

theta = np.pi/4
N = 15
x = np.sin(np.linspace(-np.pi/2, np.pi/2, 2*N))
X, Y = np.meshgrid(x, x)
f = lambda x: 1/(1 + 2*(x)**2)
Z_avg = np.zeros(X.shape)
Z_hat = np.zeros(X.shape)
for theta in np.linspace(0, 2*np.pi, 300):
    Z = f(X*np.cos(theta) + Y*np.sin(theta))
    Z_hat = fft.dctn(Z,type=1)/np.prod(Z_hat.shape)
    Z_hat[0, :] /= np.sqrt(2)
    Z_hat[:, 0] /= np.sqrt(2)
    Z_avg += np.abs(Z_hat)**2/300
Z_avg = np.sqrt(Z_avg)
im = axs[2].contour(np.arange(N), np.arange(N), np.maximum(np.log10(np.abs(Z_avg[::2, ::2])), -8))
plt.colorbar(im)

In [42]:

theta = np.pi/4
x = np.sin(np.linspace(-np.pi/2, np.pi/2, 200))
X, Y = np.meshgrid(x, x)
f = lambda x: 1/(1 + 36*(x+.1)**2)
Z = f(X*np.cos(theta) + Y*np.sin(theta))
Z_hat = fft.dctn(Z,type=1)/np.prod(Z_hat.shape)
Z_hat[0, :] /= np.sqrt(2)
Z_hat[:, 0] /= np.sqrt(2)
axs[1].imshow(np.maximum(np.log10(np.abs(Z_hat)), -12))

/tmp/ipykernel_2992990/965083437.py:9: RuntimeWarning: divide by zero encountered in log10
  axs[1].imshow(np.maximum(np.log10(np.abs(Z_hat)), -12))


In [100]:
# Stupid 3d brillouin zone plot
import numpy as np
import numpy.linalg as lin
import matplotlib.pyplot as plt


res = 14
x = np.linspace(-1, 1, res)
X, Y, Z = np.meshgrid(x, x, x)
N = 10
M = 16
p = []
tp = [np.array(i) - M//2 for i in np.ndindex((M, M, M))]

for i in np.ndindex((res, res, res)):
    print(i)
    tmp = np.array([np.array([X[i], Y[i], Z[i]]) + x for x in tp])
    j = [x for x in range(len(tp))]
    j.sort(key=lambda x: lin.norm(tmp[x]))
    ind = np.array(j[:N])
    p.extend(tmp[ind])
    
p = np.array(p)

ax = plt.figure().add_subplot(projection='3d')
ax.scatter(p[:, 0], p[:, 1], p[:, 2])
plt.show()

(0, 0, 0)
(0, 0, 1)
(0, 0, 2)
(0, 0, 3)
(0, 0, 4)
(0, 0, 5)
(0, 0, 6)
(0, 0, 7)
(0, 0, 8)
(0, 0, 9)
(0, 0, 10)
(0, 0, 11)
(0, 0, 12)
(0, 0, 13)
(0, 1, 0)
(0, 1, 1)
(0, 1, 2)
(0, 1, 3)
(0, 1, 4)
(0, 1, 5)
(0, 1, 6)
(0, 1, 7)
(0, 1, 8)
(0, 1, 9)
(0, 1, 10)
(0, 1, 11)
(0, 1, 12)
(0, 1, 13)
(0, 2, 0)
(0, 2, 1)
(0, 2, 2)
(0, 2, 3)
(0, 2, 4)
(0, 2, 5)
(0, 2, 6)
(0, 2, 7)
(0, 2, 8)
(0, 2, 9)
(0, 2, 10)
(0, 2, 11)
(0, 2, 12)
(0, 2, 13)
(0, 3, 0)
(0, 3, 1)
(0, 3, 2)
(0, 3, 3)
(0, 3, 4)
(0, 3, 5)
(0, 3, 6)
(0, 3, 7)
(0, 3, 8)
(0, 3, 9)
(0, 3, 10)
(0, 3, 11)
(0, 3, 12)
(0, 3, 13)
(0, 4, 0)
(0, 4, 1)
(0, 4, 2)
(0, 4, 3)
(0, 4, 4)
(0, 4, 5)
(0, 4, 6)
(0, 4, 7)
(0, 4, 8)
(0, 4, 9)
(0, 4, 10)
(0, 4, 11)
(0, 4, 12)
(0, 4, 13)
(0, 5, 0)
(0, 5, 1)
(0, 5, 2)
(0, 5, 3)
(0, 5, 4)
(0, 5, 5)
(0, 5, 6)
(0, 5, 7)
(0, 5, 8)
(0, 5, 9)
(0, 5, 10)
(0, 5, 11)
(0, 5, 12)
(0, 5, 13)
(0, 6, 0)
(0, 6, 1)
(0, 6, 2)
(0, 6, 3)
(0, 6, 4)
(0, 6, 5)
(0, 6, 6)
(0, 6, 7)
(0, 6, 8)
(0, 6, 9)
(0, 6, 10)
(0, 6, 11)
(0, 6, 12)
(0,

(3, 13, 11)
(3, 13, 12)
(3, 13, 13)
(4, 0, 0)
(4, 0, 1)
(4, 0, 2)
(4, 0, 3)
(4, 0, 4)
(4, 0, 5)
(4, 0, 6)
(4, 0, 7)
(4, 0, 8)
(4, 0, 9)
(4, 0, 10)
(4, 0, 11)
(4, 0, 12)
(4, 0, 13)
(4, 1, 0)
(4, 1, 1)
(4, 1, 2)
(4, 1, 3)
(4, 1, 4)
(4, 1, 5)
(4, 1, 6)
(4, 1, 7)
(4, 1, 8)
(4, 1, 9)
(4, 1, 10)
(4, 1, 11)
(4, 1, 12)
(4, 1, 13)
(4, 2, 0)
(4, 2, 1)
(4, 2, 2)
(4, 2, 3)
(4, 2, 4)
(4, 2, 5)
(4, 2, 6)
(4, 2, 7)
(4, 2, 8)
(4, 2, 9)
(4, 2, 10)
(4, 2, 11)
(4, 2, 12)
(4, 2, 13)
(4, 3, 0)
(4, 3, 1)
(4, 3, 2)
(4, 3, 3)
(4, 3, 4)
(4, 3, 5)
(4, 3, 6)
(4, 3, 7)
(4, 3, 8)
(4, 3, 9)
(4, 3, 10)
(4, 3, 11)
(4, 3, 12)
(4, 3, 13)
(4, 4, 0)
(4, 4, 1)
(4, 4, 2)
(4, 4, 3)
(4, 4, 4)
(4, 4, 5)
(4, 4, 6)
(4, 4, 7)
(4, 4, 8)
(4, 4, 9)
(4, 4, 10)
(4, 4, 11)
(4, 4, 12)
(4, 4, 13)
(4, 5, 0)
(4, 5, 1)
(4, 5, 2)
(4, 5, 3)
(4, 5, 4)
(4, 5, 5)
(4, 5, 6)
(4, 5, 7)
(4, 5, 8)
(4, 5, 9)
(4, 5, 10)
(4, 5, 11)
(4, 5, 12)
(4, 5, 13)
(4, 6, 0)
(4, 6, 1)
(4, 6, 2)
(4, 6, 3)
(4, 6, 4)
(4, 6, 5)
(4, 6, 6)
(4, 6, 7)
(4, 6, 8)
(4, 6, 9)


(7, 13, 7)
(7, 13, 8)
(7, 13, 9)
(7, 13, 10)
(7, 13, 11)
(7, 13, 12)
(7, 13, 13)
(8, 0, 0)
(8, 0, 1)
(8, 0, 2)
(8, 0, 3)
(8, 0, 4)
(8, 0, 5)
(8, 0, 6)
(8, 0, 7)
(8, 0, 8)
(8, 0, 9)
(8, 0, 10)
(8, 0, 11)
(8, 0, 12)
(8, 0, 13)
(8, 1, 0)
(8, 1, 1)
(8, 1, 2)
(8, 1, 3)
(8, 1, 4)
(8, 1, 5)
(8, 1, 6)
(8, 1, 7)
(8, 1, 8)
(8, 1, 9)
(8, 1, 10)
(8, 1, 11)
(8, 1, 12)
(8, 1, 13)
(8, 2, 0)
(8, 2, 1)
(8, 2, 2)
(8, 2, 3)
(8, 2, 4)
(8, 2, 5)
(8, 2, 6)
(8, 2, 7)
(8, 2, 8)
(8, 2, 9)
(8, 2, 10)
(8, 2, 11)
(8, 2, 12)
(8, 2, 13)
(8, 3, 0)
(8, 3, 1)
(8, 3, 2)
(8, 3, 3)
(8, 3, 4)
(8, 3, 5)
(8, 3, 6)
(8, 3, 7)
(8, 3, 8)
(8, 3, 9)
(8, 3, 10)
(8, 3, 11)
(8, 3, 12)
(8, 3, 13)
(8, 4, 0)
(8, 4, 1)
(8, 4, 2)
(8, 4, 3)
(8, 4, 4)
(8, 4, 5)
(8, 4, 6)
(8, 4, 7)
(8, 4, 8)
(8, 4, 9)
(8, 4, 10)
(8, 4, 11)
(8, 4, 12)
(8, 4, 13)
(8, 5, 0)
(8, 5, 1)
(8, 5, 2)
(8, 5, 3)
(8, 5, 4)
(8, 5, 5)
(8, 5, 6)
(8, 5, 7)
(8, 5, 8)
(8, 5, 9)
(8, 5, 10)
(8, 5, 11)
(8, 5, 12)
(8, 5, 13)
(8, 6, 0)
(8, 6, 1)
(8, 6, 2)
(8, 6, 3)
(8, 6, 4)
(8, 6

(11, 10, 12)
(11, 10, 13)
(11, 11, 0)
(11, 11, 1)
(11, 11, 2)
(11, 11, 3)
(11, 11, 4)
(11, 11, 5)
(11, 11, 6)
(11, 11, 7)
(11, 11, 8)
(11, 11, 9)
(11, 11, 10)
(11, 11, 11)
(11, 11, 12)
(11, 11, 13)
(11, 12, 0)
(11, 12, 1)
(11, 12, 2)
(11, 12, 3)
(11, 12, 4)
(11, 12, 5)
(11, 12, 6)
(11, 12, 7)
(11, 12, 8)
(11, 12, 9)
(11, 12, 10)
(11, 12, 11)
(11, 12, 12)
(11, 12, 13)
(11, 13, 0)
(11, 13, 1)
(11, 13, 2)
(11, 13, 3)
(11, 13, 4)
(11, 13, 5)
(11, 13, 6)
(11, 13, 7)
(11, 13, 8)
(11, 13, 9)
(11, 13, 10)
(11, 13, 11)
(11, 13, 12)
(11, 13, 13)
(12, 0, 0)
(12, 0, 1)
(12, 0, 2)
(12, 0, 3)
(12, 0, 4)
(12, 0, 5)
(12, 0, 6)
(12, 0, 7)
(12, 0, 8)
(12, 0, 9)
(12, 0, 10)
(12, 0, 11)
(12, 0, 12)
(12, 0, 13)
(12, 1, 0)
(12, 1, 1)
(12, 1, 2)
(12, 1, 3)
(12, 1, 4)
(12, 1, 5)
(12, 1, 6)
(12, 1, 7)
(12, 1, 8)
(12, 1, 9)
(12, 1, 10)
(12, 1, 11)
(12, 1, 12)
(12, 1, 13)
(12, 2, 0)
(12, 2, 1)
(12, 2, 2)
(12, 2, 3)
(12, 2, 4)
(12, 2, 5)
(12, 2, 6)
(12, 2, 7)
(12, 2, 8)
(12, 2, 9)
(12, 2, 10)
(12, 2, 11)
(12, 2, 

In [84]:
import numpy as np

np.arange(8)[np.array([1, 3, 4])]

array([1, 3, 4])